In [11]:
# Make predictions on data emailed by Claudio Feb 2022
# Author Peter Aug 5 2022
# The order of the parameters in each line is the usual one: alpha, cmf, cpa, pwr1par, pwr2par, pwr1perr, pwr2perr.
# For alpha and cmf, I added new values (alpha = 0, 10, 85; cmf = 2.5, 3.5, 9.5), which reflect more extreme conditions in the heliosphere: lower values of alpha and cmf correspond to a more quiet heliosphere, which result in less modulation of cosmic rays (less difference in the rigidity spectrum with respect to the local interstellar spectrum, the boundary condition at the external edge of the heliosphere), while higher values of alpha and cmf correspond to a more turbulent heliosphere, which result in more modulation.

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import numpy as np
import matplotlib.pyplot as plt
import h5py
from collections import defaultdict

import jax
import jax.numpy as jnp
from jax import random
from jax import vmap
from jax import jit
from jax import grad
assert jax.default_backend() == 'gpu'

import elegy # pip install elegy
import optax
#import flax.linen
import tensorflow_io as tfio # pip install tensorflow-io
import tensorflow as tf # Recommended not to import this with jax because will also try to grab memory.

import pandas as pd
import gcr_utils 

# Load trained model. Construct function that includes normalization of inputs and outputs.
model_path = 'model_2_256_selu_l21e-6' #'model_2_256_selu_l21e-6_do' # 'model_2_256_selu_l21e-6' #'model_2_256_selu'
f = gcr_utils.define_nn_pred(model_path, 
                             normalize_input_flag=True, 
                             denormalize_output_flag=True,
                             rebin_output_flag=False)

def load_data():
    """ Load data from Claudio."""
    filename = '../data/new_models.dat'
    # Columns: alpha, cmf, cpa, pwr1par, pwr2par, pwr1perr, pwr2perr.
    # For alpha and cmf, I added new values (alpha = 0, 10, 85; cmf = 2.5, 3.5, 9.5)
    # This reflects more extreme conditions.
    #data = np.loadtxt(filename) # Rigidity1, Rigidity2, Flux, Error
    df = pd.read_csv(filename, delimiter=' ', names=['alpha', 'cmf', 'cpa', 'pwr1par', 'pwr2par', 'pwr1perr', 'pwr2perr', 'NAN'])
    df = df.drop(columns='NAN')  # Last column has no data.
    return df
    
df = load_data()
df.head()

,alpha,cmf,cpa,pwr1par,pwr2par,pwr1perr,pwr2perr
0,0,2.5,50,0.5,0.5,0.5,0.5
1,0,2.5,50,0.5,0.5,0.5,1.1
2,0,2.5,50,0.5,0.5,0.5,1.7
3,0,2.5,50,0.5,0.5,0.5,2.3
4,0,2.5,50,0.5,0.5,1.1,0.5


In [27]:
# DEBUG: Run values through the NN model. 
x = df.iloc[-5:-1].to_numpy()
yhat = f(x)
plt.plot(yhat.T)

In [38]:
# Iterate through new dataset and produce predicted profiles (245 outputs/example)
BATCH_SIZE = 1000
tensor = tf.convert_to_tensor(df)
dataset = tfio.IODataset.from_tensor_slices(tensor).batch(BATCH_SIZE, drop_remainder=False).prefetch(tf.data.experimental.AUTOTUNE)

xs, ys, yhats = [], [], []
for i,xbatch in enumerate(dataset.as_numpy_iterator()):
    yhat = f(xbatch)
    yhats.append(yhat)
    xs.append(xbatch)
    #ys.append(ybatch)
    #break
xs = np.concatenate(xs, axis=0)
yhats = np.concatenate(yhats)
assert xs.shape[0] == yhats.shape[0]
#ys = np.concatenate(ys, axis=0)
#assert ys.shape == yhats.shape

#np.savetxt(fname=f'test_inputs_10k.csv', X=xs, fmt='%.8e')
#np.savetxt(fname=f'test_targets_10k.csv', X=ys, fmt='%.8e')
np.savetxt(fname=f'predictions.csv', X=yhats, fmt='%.8e',)

In [28]:
test_dataset = tf.data.Dataset.from_tensor_slices(df.to_numpy())


2022-08-06 09:49:32.955289: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


In [39]:
yhats.shape

(44064, 245)

In [33]:
dataset = tf.convert_to_tensor(df)
len(list(test_dataset))

100

In [35]:
test_dataset[1]

<tf.Tensor: shape=(7,), dtype=float64, numpy=array([ 0. ,  2.5, 50. ,  0.5,  0.5,  0.5,  1.1])>

In [ ]:
x = 